In [10]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

from category_encoders import TargetEncoder

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [40]:
'''
import and join test and user
write test user_join to disk 
'''
test = pd.read_csv('C:\\Users\\xzhao\\Data\\Spanish_Translation\\test_table.csv')
user = pd.read_csv('C:\\Users\\xzhao\\Data\\Spanish_Translation\\user_table.csv')
tu_join = pd.merge(test, user, on='user_id', how='left')
tu_join.to_csv('C:\\Users\\xzhao\\Data\\Spanish_Translation\\tu_join.csv', index=False)

In [134]:
otu_join = pd.read_csv('C:\\Users\\xzhao\\Data\\Spanish_Translation\\tu_join.csv')
tu_join = pd.read_csv('C:\\Users\\xzhao\\Data\\Spanish_Translation\\tu_join.csv')

In [132]:
'''
understand feature
unique count and if less than 20, unique value
Nan samples
data description
'''
def ud(df,feature):
    print('unique value count', df[feature].nunique())
    if df[feature].nunique()<20:
        print('unique value', df[feature].unique())
    print('empty records:',  df[feature].isna().sum(), '\n',df[df[feature].isna()].head(3))
    print('data description:', df[feature].describe())

In [71]:
'''
trim down irrelevant samples for the purpose of the project
    non-spanish views
    na record
    Spain users
need to test if those removed are skewed
'''

tu_join = tu_join[tu_join['browser_language']=='ES']
tu_join = tu_join[tu_join['country'].notna()]
tu_join = tu_join[tu_join['country']!='Spain']

In [73]:
print(tu_join.shape)
tu_join.head()

(334014, 12)


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1,M,21.0,Mexico
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1,M,19.0,Mexico
5,540675,2015-12-03,Direct,Mobile,ES,NaN,Android_App,0,1,F,22.0,Venezuela
8,261625,2015-12-04,Direct,Mobile,ES,NaN,Android_App,0,1,M,31.0,Bolivia
9,10427,2015-12-04,Ads,Mobile,ES,Facebook,Android_App,0,0,F,33.0,Mexico


In [136]:
ud(tu_join,'browser_language')

unique value count 1
unique value ['ES']
empty records: 0 
 Empty DataFrame
Columns: [user_id, date, source, device, browser_language, ads_channel, browser, conversion, test, sex, age, country]
Index: []
data description: count     334014
unique         1
top           ES
freq      334014
Name: browser_language, dtype: object


In [137]:
'''
test control split
'''
group_test = tu_join[tu_join['test']==1]
group_control = tu_join[tu_join['test']==0]

In [138]:
'''
sanity check
'''
print(len(group_test),len(group_control))
print(group_test['conversion'].sum(),group_control['conversion'].sum())

179598 154416
7816 7508


In [140]:
'''
define H0
calculate t and p
interperate results
'''

from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(group_control['conversion'],group_test['conversion'], equal_var=False)

print('H0: There are no significant difference between universal translation and localized translation')
print('t-statistic:', t_stat)
print('p-value:', p_value)

alpha = 0.05
if p_value > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')
    
if t_stat > 0:
    print('Control group performs better (universal translation performs better)')
else:
    print('Test group performs better (localized translation performs better)')

H0: There are no significant difference between universal translation and localized translation
t-statistic: 6.999966062026151
p-value: 2.5652349687868958e-12
Different distributions (reject H0)
Control group performs better (universal translation performs better)


# Solidify conclusion
## Control for confounding variables

In [187]:
def AB(data,column,list):
    
    '''
    control for column with criteria of list
    '''
    df = data[data[column].isin(list)]
    
    '''
    test control split
    '''
    group_test = df[df['test']==1]
    group_control = df[df['test']==0]

    '''
    sanity check
    '''
    print('T:C',len(group_test),len(group_control))
    print('T_convert:C_convert',group_test['conversion'].sum(),group_control['conversion'].sum())
    print('------------------')

    '''
    define H0
    calculate t and p
    interperate results
    '''

    from scipy.stats import ttest_ind

    t_stat, p_value = ttest_ind(group_control['conversion'],group_test['conversion'], equal_var=False)

    print('H0: There are no significant difference between universal translation and localized translation')
    print('------------------')
    print('t-statistic:', t_stat)
    print('p-value:', p_value)
    print('------------------')

    alpha = 0.05
    if p_value > alpha:
        print('Same distributions (fail to reject H0)')
    else:
        print('Different distributions (reject H0)')

    if t_stat > 0:
        print('Control group performs better (universal translation performs better)')
    else:
        print('Test group performs better (localized translation performs better)')
    
    return t_stat
    return p_value

In [193]:
for i in otu_join.columns.tolist():
    if otu_join[i].nunique()<20:
        print (i,otu_join[i].unique().tolist())

date ['2015-12-03', '2015-12-04', '2015-11-30', '2015-12-01', '2015-12-02']
source ['Direct', 'Ads', 'SEO']
device ['Web', 'Mobile']
browser_language ['ES', 'Other', 'EN']
ads_channel [nan, 'Google', 'Facebook', 'Bing', 'Yahoo', 'Other']
browser ['IE', 'Chrome', 'Android_App', 'FireFox', 'Iphone_App', 'Safari', 'Opera']
conversion [1, 0]
test [0, 1]
sex ['M', 'F', nan]
country ['Spain', 'Mexico', 'Venezuela', 'Bolivia', 'Colombia', 'Uruguay', 'El Salvador', 'Nicaragua', 'Peru', 'Costa Rica', 'Chile', 'Argentina', 'Ecuador', 'Guatemala', 'Honduras', 'Paraguay', 'Panama', nan]


In [188]:
AB(tu_join,'country',['Paraguay'])

T:C 3082 3007
T_convert:C_convert 145 142
------------------
H0: There are no significant difference between universal translation and localized translation
------------------
t-statistic: 0.03235194731360059
p-value: 0.9741924443436722
------------------
Same distributions (fail to reject H0)
Control group performs better (universal translation performs better)


0.03235194731360059

In [191]:
'''
what im trying to do here is to find those training samples (defined by column name and value of that sample under the column)
that have a p value > alpha 

if there are no such samples or these samples consists a small slice of the original data frame, i can  feel confident about AB test's
conclusion
'''

for i in otu_join.columns.tolist():
    for j in otu_join[i].unique():
        print(AB(otu_join,i,j))
#         if AB(otu_join,i,j).p_value>0.05:
#             print (i,j,AB(otu_join,i,j).p_value,'fail to reject H0')

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [int64]